In [1]:
# Mounting Drive for getting dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
# build a CNN model with 3 hidden layers for Food-101 dataset
def build_model():
    pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='max'
    )

    pretrained_model.trainable = False
    inputs = pretrained_model.input

    x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
    x = tf.keras.layers.Dense(128, activation='relu')(x)

    outputs = tf.keras.layers.Dense(9, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import os
import glob as gb

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
train_df = '/content/drive/MyDrive/Capstone_Project/Data/Train'
test_df = '/content/drive/MyDrive/Capstone_Project/Data/Test'

In [5]:
for folder in os.listdir(train_df):
    files = gb.glob(pathname= str(train_df+ '/'+ folder + '/*.jpg'))
    print(f'For training data, found {len(files)} in folder {folder}')

For training data, found 7 in folder Puri
For training data, found 7 in folder Bread
For training data, found 7 in folder Ice Cream
For training data, found 7 in folder Dosa
For training data, found 7 in folder Pizza
For training data, found 7 in folder Chappati
For training data, found 7 in folder Maggi
For training data, found 7 in folder Idly
For training data, found 7 in folder Pongal


In [6]:
for folder in os.listdir(test_df):
    files = gb.glob(pathname= str(test_df+ '/'+ folder + '/*.jpg'))
    print(f'For testing data, found {len(files)} in folder {folder}')

For testing data, found 3 in folder Bread
For testing data, found 3 in folder Idly
For testing data, found 3 in folder Pizza
For testing data, found 3 in folder Pongal
For testing data, found 3 in folder Puri
For testing data, found 3 in folder Ice Cream
For testing data, found 3 in folder Dosa
For testing data, found 3 in folder Maggi
For testing data, found 3 in folder Chappati


In [7]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [8]:
train_generator = train_datagen.flow_from_directory(
    train_df,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=16
)

Found 63 images belonging to 9 classes.


In [9]:
model = build_model()
# train model
history = model.fit(
    train_generator,
    epochs=20
)

9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/20
4/4 [==============================] - 22s 4s/step - loss: 3.8178 - accuracy: 0.2540
Epoch 2/20
4/4 [==============================] - 2s 411ms/step - loss: 2.0623 - accuracy: 0.3968
Epoch 3/20
4/4 [==============================] - 2s 418ms/step - loss: 0.7807 - accuracy: 0.7460
Epoch 4/20
4/4 [==============================] - 2s 423ms/step - loss: 0.3778 - accuracy: 0.8730
Epoch 5/20
4/4 [==============================] - 2s 429ms/step - loss: 0.0858 - accuracy: 1.0000
Epoch 6/20
4/4 [==============================] - 2s 411ms/step - loss: 0.0356 - accuracy: 1.0000
Epoch 7/20
4/4 [==============================] - 2s 407ms/step - loss: 0.0250 - accuracy: 1.0000
Epoch 8/20
4/4 [==============================] - 2s 408ms/step - loss: 0.0124 - accuracy: 1.0000
Epoch 9/20
4/4 [==============================] - 2s 403ms/step - loss: 0.0070 - accuracy: 1.0000
Epoch 10/20
4/4 [==============================] - 2s 421

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [13]:
model.save('/content/drive/MyDrive/Capstone_Project/Models/MobileNet_v2.h5')

In [11]:
test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_directory(
    test_df,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=16
)

Found 27 images belonging to 9 classes.


In [12]:
results = model.evaluate(test_generator)
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

2/2 [==============================] - 5s 4s/step - loss: 0.9550 - accuracy: 0.7407
Test Accuracy: 74.07%
